In [1]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import NBEATSModel
from sklearn.metrics import mean_squared_error, mean_squared_log_error
import matplotlib.pyplot as plt
import torch

C:\Users\Usman.nazir\.conda\envs\pytorch_env\lib\site-packages\statsforecast\utils.py:237: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),


In [2]:
torch.set_float32_matmul_precision('medium')

In [3]:
df = pd.read_csv("train_darts_univariate.csv", parse_dates=["date"])

In [4]:
df['sales'] = df['sales'].astype('float32')

In [5]:
df.head()

,date,store_nbr,family,sales
0,2013-01-01,1,0,0.0
1,2013-01-01,1,1,0.0
2,2013-01-01,1,2,0.0
3,2013-01-01,1,3,0.0
4,2013-01-01,1,4,0.0


In [6]:
unique_combinations = df[['store_nbr', 'family']].drop_duplicates()

In [7]:
total_rmse = 0
total_rmsle = 0
num_models = 0

In [8]:
trained_models = {}

In [9]:
for index, row in unique_combinations.iterrows():
    store_nbr = row['store_nbr']
    family = row['family']
    
    print(f"\nTraining model for store {store_nbr} and family {family}...")
    
    df_subset = df[(df['store_nbr'] == store_nbr) & (df['family'] == family)]
    
    ts = TimeSeries.from_dataframe(df_subset, value_cols=["sales"])

    split_ratio = 0.8
    training_size = int(len(ts) * split_ratio)
    train = ts[:training_size]
    val = ts[training_size:]

    # pl_trainer_kwargs = {"accelerator": "cpu"}

    model = NBEATSModel(
        # pl_trainer_kwargs=pl_trainer_kwargs,
        input_chunk_length=7,
        output_chunk_length=7,
        n_epochs=1,
        activation='LeakyReLU'
    )
    
    model.fit(train)

    trained_models[(store_nbr, family)] = model

    predictions = model.predict(n=len(val))

    val = val.pd_series().tolist()
    predictions = predictions.pd_series().tolist()

    predictions = np.maximum(predictions, 0)

    rmse = np.sqrt(mean_squared_error(val, predictions))
    rmsle = np.sqrt(mean_squared_error(np.log1p(val), np.log1p(predictions)))

    total_rmse += rmse
    total_rmsle += rmsle
    num_models += 1

    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"Root Mean Squared Logarithmic Error (RMSLE): {rmsle}")

    plt.figure(figsize=(12, 6))
    plt.plot(val, label='Actual')
    plt.plot(predictions, label='Predictions')
    plt.title(f"Model for store {store_nbr} and family {family}")
    plt.legend()
    plt.savefig(f"plots/M10.2/M10.2_store{store_nbr}_family{family}.png")
    plt.close()


Training model for store 1 and family 0...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 6.1 M 
---------------------------------------------------
6.1 M     Trainable params
1.3 K     Non-trainable params
6.1 M     Total params
24.249    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 6.1 M 
---------------------------------------------------
6.1 M     Trainable params
1.3 K     Non-trainable params
6.1 M     Total params
24.249    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

Root Mean Squared Error (RMSE): 9.953594475922408
Root Mean Squared Logarithmic Error (RMSLE): 0.6335573433010318


In [10]:
avg_rmse = total_rmse / num_models
avg_rmsle = total_rmsle / num_models

In [11]:
print(f"Average Root Mean Squared Error (RMSE) across all models: {avg_rmse}")
print(f"Average Root Mean Squared Logarithmic Error (RMSLE) across all models: {avg_rmsle}")

Average Root Mean Squared Error (RMSE) across all models: 681939.4291425318
Average Root Mean Squared Logarithmic Error (RMSLE) across all models: 1.1914348169161166
